# Trend Lines and Channels Detection

In [ ]:
import ccxt
import pandas as pd

In [11]:


# exchange = ccxt.binance()
# exchange.load_markets()

# '''symbols variabl contains all the symbols of the exchange '''
# symbols = exchange.symbols

# # ticker = 'BTC-USDT'
# # timeframe ='4h'
# def ccxt_data(symbol='HNT/USDT', timeframe ='4h', limit=9000):

#     """ use global to make a variable accesible by oher functions
#     and always declare it before using it  """
#     global df

#     '''if you dont do timeframe=timeframe it will not work 
#     but in symbol it works !!'''
#     # fetch returns a list
#     bars = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

#     df = pd.DataFrame(bars[:], columns=['time', 'open', 'high', 'low', 'close', 'volume'])
#     # Time is in unix format
#     df['time'] = pd.to_datetime(df['time'], unit='ms')
#     df.set_index('time', inplace=True)
    
#     return df


# df = pd.read_csv('C:\\Users\Grant\Desktop\work_git\work\data\\1d\\ATOMUSDT_1d.csv', index_col='Time')




In [28]:
def find_csv(pair = 'btc',tf = '4h'):

    pair.upper()
    # data=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\{tf}\{pair}USDT_{tf}.csv", parse_dates=True)
    df=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\{tf}\{pair}USDT_{tf}.csv", parse_dates=True, index_col='Time')
    return df

find_csv('atom', '4h')



,Open,High,Low,Close,Volume
Time,,,,,
2021-09-15 00:00:00,34.19,35.35,32.61,33.08,1254084.34
2021-09-15 04:00:00,33.08,34.39,32.78,34.19,923179.36
2021-09-15 08:00:00,34.21,34.41,33.00,33.34,835751.52
2021-09-15 12:00:00,33.35,33.76,32.71,33.36,835291.11
2021-09-15 16:00:00,33.37,35.05,32.98,33.98,1212385.71
...,...,...,...,...,...
2022-02-27 16:00:00,28.96,28.99,27.20,27.31,572346.16
2022-02-27 20:00:00,27.30,27.69,26.28,26.77,1046045.00
2022-02-28 00:00:00,26.77,28.19,26.38,27.46,859931.57


In [12]:

#Check if NA values are in data
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,Open,High,Low,Close,Volume
0,29.84,29.86,28.72,29.00,86596.94
1,28.98,29.64,28.72,29.58,94484.78
2,29.58,29.69,29.16,29.61,66852.30
3,29.61,30.61,29.38,29.81,213040.10
4,29.80,30.19,29.65,29.79,66492.37
5,29.79,30.79,29.73,30.46,69780.49
6,30.46,33.74,29.49,32.40,789770.40
7,32.40,33.44,31.94,32.48,235338.25
8,32.47,32.60,31.58,31.61,124020.56
9,31.59,31.96,30.54,31.26,117164.06


In [13]:
dfpl = df[:]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [14]:
import numpy as np
from matplotlib import pyplot
backcandles= 50
wind = 5

candleid = 880

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])
for i in range(candleid-backcandles, candleid+1, wind):
    minim = np.append(minim, df.Low.iloc[i:i+wind].min())
    xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
for i in range(candleid-backcandles, candleid+1, wind):
    maxim = np.append(maxim, df.High.loc[i:i+wind].max())
    xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)

dfpl = df[candleid-backcandles:candleid+backcandles]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))

# Fitting intercepts to meet highest or lowest candle point in time slice

In [15]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to meet highest or lowest candle point in time slice
adjintercmin = df.Low.loc[candleid-backcandles:candleid].min() - slmin*df.Low.iloc[candleid-backcandles:candleid].idxmin()
adjintercmax = df.High.loc[candleid-backcandles:candleid].max() - slmax*df.High.iloc[candleid-backcandles:candleid].idxmax()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.show()

# Fitting intercepts to wrap price candles

In [16]:
dfpl = df[candleid-wind-backcandles:candleid+backcandles]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

#-------------------------------------------------------------------------
# Fitting intercepts to wrap price candles
adjintercmax = (df.High.iloc[xxmax] - slmax*xxmax).max()
adjintercmin = (df.Low.iloc[xxmin] - slmin*xxmin).min()
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))
fig.show()

# backcandles time window more dynamic

In [17]:
import numpy as np
from matplotlib import pyplot
backcandles= 40 # 6*8
brange = 10 # backcandles//4 #should be less than backcandles
wind = 6

candleid = 500

optbackcandles= backcandles
sldiff = 999

for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.Low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.High.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    if(abs(slmin-slmax)<sldiff):
        sldiff = abs(slmin-slmax)
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

adjintercmax = (df.High.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.Low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.show()

30
31
32
33
34
36
37
38
39
43


In [18]:
import numpy as np
from matplotlib import pyplot
backcandles= 88
brange = 50 #should be less than backcandles
wind = 5

candleid = 700

optbackcandles= backcandles
sldiff = 10
sldist = 55
for r1 in range(backcandles-brange, backcandles+brange):
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    
    for i in range(candleid-r1, candleid+1, wind):
        minim = np.append(minim, df.Low.iloc[i:i+wind].min())
        xxmin = np.append(xxmin, df.Low.iloc[i:i+wind].idxmin())
    for i in range(candleid-r1, candleid+1, wind):
        maxim = np.append(maxim, df.High.loc[i:i+wind].max())
        xxmax = np.append(xxmax, df.High.iloc[i:i+wind].idxmax())
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    
    dist = (slmax*candleid + intercmax)-(slmin*candleid + intercmin)
    if(dist<sldist): #abs(slmin-slmax)<sldiff and
        #sldiff = abs(slmin-slmax)
        sldist = dist
        optbackcandles=r1
        slminopt = slmin
        slmaxopt = slmax
        intercminopt = intercmin
        intercmaxopt = intercmax
        maximopt = maxim.copy()
        minimopt = minim.copy()
        xxminopt = xxmin.copy()
        xxmaxopt = xxmax.copy()

        
print(optbackcandles)
dfpl = df[candleid-wind-optbackcandles-backcandles:candleid+optbackcandles]  
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

adjintercmax = (df.High.iloc[xxmaxopt] - slmaxopt*xxmaxopt).max()
adjintercmin = (df.Low.iloc[xxminopt] - slminopt*xxminopt).min()
fig.add_trace(go.Scatter(x=xxminopt, y=slminopt*xxminopt + adjintercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmaxopt, y=slmaxopt*xxmaxopt + adjintercmax, mode='lines', name='max slope'))
fig.show()

78
